In [69]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")

key = x_test['ID']
x_test = x_test.drop('ID', axis=1)
y_train = y_train['Reached.on.Time_Y.N']
x_train = x_train.drop('ID', axis=1)

concat = pd.concat([x_train, x_test], axis=0)
obj = ['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls', 'Product_importance', 'Gender']
concat['Gender'] = concat['Gender'].map({'M':True, 'F':False})
concat.loc[concat['Customer_care_calls'] == '$7', 'Customer_care_calls'] = '7'
concat['Customer_care_calls'] = concat['Customer_care_calls'].astype(int)
target = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance']
concat = pd.get_dummies(concat, columns = target)
x_train = concat.iloc[:len(x_train), :]
x_test = concat.iloc[len(x_train):, :]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

model = RandomForestClassifier(
    n_estimators = 500,
    max_depth = 15,
    min_samples_split = 2,
    min_samples_leaf = 2,
    random_state = 42
)

model.fit(x_t, y_t)
pred = model.predict(x_va)
f1 = f1_score(y_va, pred)
print('F1 Score: ', f1)

submit = pd.DataFrame()
submit['ID'] = key
submit['Reached.on.Time_Y.N'] = model.predict(x_test)
display(submit)
#to_csv.submit('result.csv', index=False)

F1 Score:  0.6856309263311452


,ID,Reached.on.Time_Y.N
0,6811,1
1,4320,0
2,5732,0
3,7429,1
4,2191,1
...,...,...
4396,2610,1
4397,3406,0
4398,10395,0
4399,3646,0


In [93]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')

key = test['ID']
x_train = train.drop(['ID', 'TravelInsurance'], axis=1)
y_train = train['TravelInsurance']
x_test = test.drop('ID', axis=1)
concat = pd.concat([x_train, x_test], axis=0)
concat['Employment Type'] = concat['Employment Type'].map({'Private Sector/Self Employed':True, 'Government Sector':False})
target = ['GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
for col in target:
    concat[col] = concat[col].map({'Yes':True, 'No':False})

x_train = concat.iloc[:len(x_train),:]
x_test = concat.iloc[len(x_train):,:]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

model = RandomForestClassifier(
    n_estimators = 500,
    random_state = 42,
    max_depth = 15,
    min_samples_leaf = 4,
    min_samples_split = 5
)
model.fit(x_t, y_t)
pred = model.predict_proba(x_va)[:, -1]
auc = roc_auc_score(y_va, pred)
print('AUC: ', auc)

AUC:  0.7957147277227723
